In [1]:
import pandas as pd 
import os
import math 
import matplotlib.pyplot as plt
import random 
from sentence_transformers import SentenceTransformer 
from collections import Counter
from umap import UMAP
from sklearn.neighbors import KernelDensity
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import KernelDensity
import math
import random
import pickle
from threading import Thread 
import uuid
import string

/Users/peetiphartsuebpeng/Documents/CONDA_ENV/env-wizmap/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


# Embed

In [ ]:
df = pd.read_parquet('../../../08-HERB/input/herb_researchgate_10keyword.parquet')
df

In [ ]:
df = df[~df['abstract'].isna()]

In [ ]:
df

In [ ]:
df['data_process'] = 'title: ' + df['title'] + ' pub_year: ' + df['pub_year'] + ' keyword: ' + df['keyword'] + ' abstract: ' + df['abstract']
df

In [ ]:
embed = SentenceTransformer('all-MiniLM-L6-v2', device='cpu')
vector = embed.encode(df['data_process'].tolist(), show_progress_bar=True, batch_size=126)

In [ ]:
df['vector'] = [i for i in vector]

In [ ]:
os.makedirs('./output', exist_ok=True)
df.to_parquet('./output/df.parquet')

In [ ]:
df

# optmz

In [2]:
df = pd.read_parquet('./output/df.parquet')
df

,id,title,abstract,pub_year,authors_list,keyword,relative_date,link,data_process,vector
0,Stemona_collinsiae_1,Quantitative HPLC Determination and Stability ...,A reversed-phase high-performance liquid chrom...,2008,"[Ruedeekorn Wiwattanapatapee, Chitchamai Ovatl...",Stemona_collinsiae,2024-06-01T14:27:30.335303,https://www.researchgate.net/publication/22577...,title: Quantitative HPLC Determination and Sta...,"[-0.0074133547, -0.017918514, -0.08736931, -0...."
1,Stemona_collinsiae_2,Insecticidal activity of Stemona collinsiae ro...,"In Thai indigenous knowledge, Stemona plant ha...",2017,"[Siriluck Attrapadung, Aurapa Sakulpanich, Wan...",Stemona_collinsiae,2024-06-01T14:27:54.172213,https://www.researchgate.net/publication/31713...,title: Insecticidal activity of Stemona collin...,"[0.0048623416, -0.044462435, -0.1135353, -0.04..."
2,Stemona_collinsiae_3,Oral toxicity of various Stemona collinsiae cr...,Stemona collinsiae exhibits insecticidal resis...,2021,"[Nontapat Sarovath, Preeyanate Dathong, Napaso...",Stemona_collinsiae,2024-06-01T14:28:11.782212,https://www.researchgate.net/publication/35456...,title: Oral toxicity of various Stemona collin...,"[0.053052347, -0.042213105, -0.082500555, -0.0..."
3,Stemona_collinsiae_4,Optimized extraction for high yield of insecti...,"Stemona plants, which are popularly called “No...",2013,"[Sumet Kongkiatpaiboon, Phyto Product Research...",Stemona_collinsiae,2024-06-01T14:28:29.777971,https://www.researchgate.net/publication/23615...,title: Optimized extraction for high yield of ...,"[0.00866904, 0.013025655, -0.08530998, -0.0659..."
4,Stemona_collinsiae_5,Acetylcholinesterase inhibitory activity of di...,Introduction: Acetylcholinesterase (AChE) inhi...,2013,"[Pattarajinda Virote, Khon Kaen University, Wa...",Stemona_collinsiae,2024-06-01T14:28:51.335262,https://www.researchgate.net/publication/24503...,title: Acetylcholinesterase inhibitory activit...,"[0.01852814, -0.03028386, -0.07568237, -0.0225..."
...,...,...,...,...,...,...,...,...,...,...
2138,Allium_sativum_237,Effects of Onion (Allium cepa L.) and Garlic (...,In the present study the effects of individual...,2012,"[Sunčica Kocić-Tanackov, University of Novi Sa...",Allium_sativum,2024-06-03T20:50:46.716683,https://www.researchgate.net/publication/22400...,title: Effects of Onion (Allium cepa L.) and G...,"[-0.03210118, -0.026279455, -0.050988115, 0.03..."
2139,Allium_sativum_238,Methods of Isolation of Active Substances from...,Garlic (Allium sativum L.) is widely used in t...,2022,"[Monika Bar, Urszula E. Binduga, Wyższa Szkoła...",Allium_sativum,2024-06-03T20:51:06.279667,https://www.researchgate.net/publication/36187...,title: Methods of Isolation of Active Substanc...,"[-0.04116749, -0.033044282, -0.09556768, 0.057..."
2142,Allium_sativum_241,Garlic (Allium sativum L.): a potential unique...,Coronavirus disease 2019 (COVID-19) is the cur...,2020,"[Sucheta Khubber, Seyed Mohammad Taghi Gharibz...",Allium_sativum,2024-06-03T20:53:44.404606,https://www.researchgate.net/publication/34705...,title: Garlic (Allium sativum L.): a potential...,"[-0.061984845, 0.005201916, -0.10571754, 0.041..."
2143,Allium_sativum_242,Character association and path analysis in gar...,Character association and path analysis in twe...,2014,"[Shiv Lal, NA Ahmed, S. R Singh, National Rese...",Allium_sativum,2024-06-03T20:53:56.664256,https://www.researchgate.net/publication/26982...,title: Character association and path analysis...,"[0.002141517, -0.003871272, -0.04897455, 0.057..."


In [4]:
import warnings
warnings.filterwarnings('ignore')

def generate_random_uid(length=5):
    characters = str(uuid.uuid4())
    uid = ''.join(random.choices(characters, k=length))
    return uid

def Probability(data):
    frequency = Counter(data)
    total_count = sum(frequency.values())
    probability = {char: count / total_count for char, count in frequency.items()}
    
    return probability

def Entropy(data):
    probability = Probability(data)
    entropy = -sum(p * math.log2(p) for p in probability.values())
    
    return entropy

# def plot_Density(umap_model, bandwidth: float = 0.02, ax=None):

#     xs = umap_model.embedding_[:,0]
#     ys = umap_model.embedding_[:,1]
#     X_umap = np.stack((xs, ys), axis=1)
#     # Kernel Density Estimation
#     kde = KernelDensity(bandwidth=bandwidth)
#     kde.fit(X_umap)
#     # Create a meshgrid to plot KDE result
#     x_min, x_max = X_umap[:, 0].min() - 1, X_umap[:, 0].max() + 1
#     y_min, y_max = X_umap[:, 1].min() - 1, X_umap[:, 1].max() + 1
#     xx, yy = np.meshgrid(np.linspace(x_min, x_max, 500), np.linspace(y_min, y_max, 500))
#     Z = np.exp(kde.score_samples(np.c_[xx.ravel(), yy.ravel()]))
#     Z = Z.reshape(xx.shape)
    
#     # Plot KDE result on the given axis
#     if ax is None:
#         plt.figure(figsize=(12,6))
#         ax = plt.gca()
        
#     contour = ax.contourf(xx, yy, Z, cmap=plt.cm.Blues)
#     ax.scatter(xs, ys, alpha=0.2, s=0.1)
    
#     entropy = Entropy(Z.reshape((-1,1)).flatten())
#     params = umap_model.get_params()
#     ax.set_title(f'Entropy Scores: {round(entropy,4)}')
#     ax.set_xlabel('UMAP Dimension 1')
#     ax.set_ylabel('UMAP Dimension 2')

#     return Z.reshape((-1,1)), Z

def get_Density(umap_model, max_sample=100000, SEED=1234, grid_size=200):
    
    xs = umap_model.embedding_[:,0]
    ys = umap_model.embedding_[:,1]
    projected_emb = np.stack((xs, ys), axis=1)

    x_min, x_max = np.min(xs), np.max(xs)
    y_min, y_max = np.min(ys), np.max(ys)
    
    x_gap = x_max - x_min
    y_gap = y_max - y_min
    
    if x_gap > y_gap:
        # Expand the larger range to leave some padding in the plots
        x_min -= x_gap / 50
        x_max += x_gap / 50
        x_gap = x_max - x_min
        
        # Regulate the 2D grid to be a square
        y_min -= (x_gap - y_gap) / 2
        y_max += (x_gap - y_gap) / 2
    else:
        # Expand the larger range to leave some padding in the plots
        y_min -= y_gap / 50
        y_max += y_gap / 50
        y_gap = y_max - y_min
        
        # Regulate the 2D grid to be a square
        x_min -= (y_gap - x_gap) / 2
        x_max += (y_gap - x_gap) / 2
    
    # Estimate on a 2D grid
    grid_xs = np.linspace(x_min, x_max, grid_size)
    grid_ys = np.linspace(y_min, y_max, grid_size)
    xx, yy = np.meshgrid(grid_xs, grid_ys)
    
    grid = np.vstack([xx.ravel(), yy.ravel()]).transpose()

    sample_size = min(max_sample, len(xs))
    n = sample_size
    d = projected_emb.shape[1]
    bw = (n * (d + 2) / 4.)**(-1. / (d + 4))
    
    # We use a random sample to fit the KDE for faster run time
    rng = np.random.RandomState(SEED)
    random_indexes = rng.choice(range(projected_emb.shape[0]),
                                min(projected_emb.shape[0], sample_size),
                                replace=False)
    
    kde = KernelDensity(kernel='gaussian', bandwidth=bw)
    kde.fit(projected_emb[random_indexes, :])

    log_density = kde.score_samples(grid)
    log_density = np.exp(log_density)
    grid_density = np.reshape(log_density, xx.shape)
    
    # cfset = ax.contourf(xx, yy, grid_density.round(4),
    #                     levels=np.linspace(0, np.max(grid_density), 20),
    #                     cmap='Blues',
    #                     alpha=1)
    
    return grid_density, xx, yy 

def perform_grid_search(n_neighbors, min_dist, spread, vector, grid_size, save_path):
    
    random_number = ''.join(map(str, random.sample(range(1, 100), 4)))
    umap_model = UMAP(n_neighbors=n_neighbors, 
      n_components=2, 
      min_dist=min_dist, 
      metric='cosine',
      verbose=False, 
      spread=spread, 
      random_state=int(random_number)) 
    
    umap_model.fit_transform(vector)
    grid_density, xx, yy = get_Density(umap_model = umap_model, grid_size=grid_size)

    model_idx = generate_random_uid(6)
    model_rs = {'model_index': model_idx, 
        'model': umap_model, 
        'density':{'grid_density':grid_density, 'xx':xx, 'yy':yy}}
    
    with open(f'{save_path}/umap-{int(random_number)}-{model_idx}.pkl', 'wb') as file:
        pickle.dump(model_rs, file)

def grid_search_umap(params, 
        vector, 
        grid_size=200,
        save_path='./output/model_log'):

    list_of_result = []
    threads = []
    
    n_neighbors_list = params.get('n_neighbors')
    min_dist_list = params.get('min_dist')
    spread_list = params.get('spread')

    os.makedirs(save_path, exist_ok=True)
    total_iterations = len(n_neighbors_list) * len(min_dist_list) * len(spread_list)
    progress_bar = tqdm(total=total_iterations, desc="Processing")

    for i, n_neighbors in enumerate(n_neighbors_list):
        for j, min_dist in enumerate(min_dist_list):
            for k, spread in enumerate(spread_list):
                perform_grid_search(n_neighbors, min_dist, spread, vector, grid_size, save_path)
                progress_bar.update(1)

In [5]:
params = {'n_neighbors': [2,3,5,10,20,40,80,100], # 8
          'min_dist': [0.001, 0.01, 0.1, 0.15, 0.2, 0.4],  # 6
          'spread': [0.4,0.75, 0.8, 1, 4,]} # 5

grid_search_umap(params, df['vector'].tolist(), 200, './output/model_log4/')

Processing:   0%|      | 0/240 [00:00<?, ?it/s]OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


TypeError: 'float' object cannot be interpreted as an integer

In [4]:
# params = {'n_neighbors': [2,3,5,10,20,40,80,100], # 8
#           'min_dist': [0.001, 0.01, 0.1, 0.15, 0.2, 0.4],  # 6
#           'spread': [0.4,0.75, 0.8, 1, 4,]} # 5

# grid_search_umap(params, df['vector'].tolist(), 800, './output/model_log3/')

Processing: 100%|████████████████████| 240/240 [1:55:02<00:00, 28.76s/it]


In [ ]:
# params = {'n_neighbors': [3,20,80], 
#           'min_dist': [0.001,0.15, 0.4], 
#           'spread': [0.4,0.8, 4,]}

# result = grid_search_umap(params, vector=df['vector'].tolist(),

In [ ]:

# params = {'n_neighbors': [3,20,80], 
#           'min_dist': [0.001,0.15, 0.4], 
#           'spread': [0.4,0.8, 4,]}

# result = grid_search_umap(params, vector=df['vector'].tolist(),)

In [36]:
def load_pickle(path): 
    with open(f'{path}', 'rb') as file : 
        rs = pickle.load(file)
    return rs

In [37]:
data = load_pickle('./output/model_log3/umap-1062485-fb1b86.pkl')

In [38]:
data.keys()

dict_keys(['model_index', 'model', 'density'])

In [39]:
density = data.get('density').get('grid_density').reshape(-1,1).flatten()
density

array([5.61998857e-10, 7.76218746e-10, 1.06842825e-09, ...,
       5.19685693e-48, 2.29691388e-48, 1.01209536e-48])

In [41]:
hist, bin_a = np.histogram(density, bins=100)
p=hist/np.sum(hist)

In [46]:
-np.sum(p*np.log2(p))

3.774357690483211

In [44]:
Entropy(p)

6.53630731475309

In [30]:
Probability(density)

{5.619988565408944e-10: 1.5625e-06,
 7.762187464022079e-10: 1.5625e-06,
 1.068428246278968e-09: 1.5625e-06,
 1.4656140946102786e-09: 1.5625e-06,
 2.003583584658115e-09: 1.5625e-06,
 2.729665326958122e-09: 1.5625e-06,
 3.7061758328332e-09: 1.5625e-06,
 5.01484960481574e-09: 1.5625e-06,
 6.762475581613183e-09: 1.5625e-06,
 9.08803659243121e-09: 1.5625e-06,
 1.217171212943927e-08: 1.5625e-06,
 1.6246179494797244e-08: 1.5625e-06,
 2.1610735476184455e-08: 1.5625e-06,
 2.8648861369328192e-08: 1.5625e-06,
 3.784996951269621e-08: 1.5625e-06,
 4.983620046250437e-08: 1.5625e-06,
 6.539528717244617e-08: 1.5625e-06,
 8.552066631778202e-08: 1.5625e-06,
 1.1146019695913274e-07: 1.5625e-06,
 1.4477504214543385e-07: 1.5625e-06,
 1.8741047803675572e-07: 1.5625e-06,
 2.4178061489890793e-07: 1.5625e-06,
 3.1086924089595754e-07: 1.5625e-06,
 3.983492276138955e-07: 1.5625e-06,
 5.087231587616112e-07: 1.5625e-06,
 6.47488051875117e-07: 1.5625e-06,
 8.2132722663096e-07: 1.5625e-06,
 1.0383325201807953e-06: 1

In [ ]:
Probability(density)

In [29]:
entp = Entropy(density)
entp

19.28771237954945

In [ ]:
# def Probability(data):
#     frequency = Counter(data)
#     total_count = sum(frequency.values())
#     probability = {char: count / total_count for char, count in frequency.items()}
    
#     return probability
    
# def Entropy(data):
#     probability = Probability(data)
#     entropy = -sum(p * math.log2(p) for p in probability.values())
    
#     return entropy

# def get_Density(xs, ys, bandwidth: int = 0.2):
#     X_umap = np.stack((xs, ys), axis=1)
#     # Kernel Density Estimation
#     kde = KernelDensity(bandwidth=bandwidth)
#     kde.fit(X_umap)
#     # Create a meshgrid to plot KDE result
#     x_min, x_max = X_umap[:, 0].min() - 1, X_umap[:, 0].max() + 1
#     y_min, y_max = X_umap[:, 1].min() - 1, X_umap[:, 1].max() + 1
#     xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100), np.linspace(y_min, y_max, 100))
#     Z = np.exp(kde.score_samples(np.c_[xx.ravel(), yy.ravel()]))
#     Z = Z.reshape(xx.shape)
#     # Plot KDE result
#     plt.figure(figsize=(12,6))
#     plt.contourf(xx, yy, Z, cmap=plt.cm.Blues)
#     plt.colorbar()
#     # Plot center points
#     plt.scatter(xs, ys, alpha=0.2, s=0.1)
#     entropy = Entropy(Z.reshape((-1,1)).flatten())
#     plt.title(f'Entropy Scores: {round(entropy,4)}')
#     plt.xlabel('UMAP Dimension 1')
#     plt.ylabel('UMAP Dimension 2')
#     plt.show() 

#     return Z.reshape((-1,1))

# def grid_search_umap(params, vector, return_output = False):
    
#     model_idx = 0
#     list_of_result = []
#     n_neighbors_list = params.get('n_neighbors')
#     min_dist_list = params.get('min_dist')
#     spread_list = params.get('spread')
#     bandwidth = params.get('bandwidth')
    
#     total_iterations = len(n_neighbors_list) * len(min_dist_list) * len(spread_list)
#     progress_bar = tqdm(total=total_iterations, desc="Processing")
    
#     for i, n_neighbors in enumerate(n_neighbors_list):
#         for j, min_dist in enumerate(min_dist_list):
#             for k, spread in enumerate(spread_list):
#                 random_number = ''.join(map(str, random.sample(range(1, 100), 4)))
#                 umap_model = UMAP(n_neighbors=n_neighbors, 
#                                   n_components=2, 
#                                   min_dist=min_dist, 
#                                   metric='cosine',
#                                   verbose=False, 
#                                   spread=spread, 
#                                   random_state=int(random_number)) 
#                 umap_model.fit_transform(vector)

#                 if return_output :
#                     list_of_result.append({'model_index': model_idx,'model': umap_model})
                    
#                 tmp = get_Density(xs = umap_model.embedding_[:,0], ys = umap_model.embedding_[:,1])
#                 tmp = None
#                 model_idx = model_idx + 1
#                 project_emb_umap = None
#                 umap_model = None
#                 progress_bar.update(1)

#     return list_of_result